In [ ]:
# File: needle.ipynb
# Code: Claude Code and Codex
# Review: Ryoichi Ando (ryoichi.ando@zozo.com)
# License: Apache v2.0

In [ ]:
import random

from frontend import App

# create an app
app = App.create("needle")

# create a square sheet mesh
V, F = app.mesh.square(res=129)
app.asset.add.tri("sheet", V, F)

# create a cone mesh to act as the needle
V, F = app.mesh.cone(height=5, Nr=40, Ny=64, Nb=12, radius=1, sharpen=2)
app.asset.add.tri("cone", V, F)

# create a sphere tetrahedral mesh as the weight
V, F, T = app.mesh.icosphere(r=0.25, subdiv_count=4).tetrahedralize()
app.asset.add.tet("sphere", V, F, T)

# create a scene
scene = app.scene.create()

# add cone needle pointing upward
scene.add("cone").at(0, -0.25, 0).scale(0.2).rotate(90, "x").pin()

# add heavy sphere that will push sheets down onto needle
sphere = scene.add("sphere").at(0, 1, 0)
sphere.param.set("density", 1e5)
sphere.pin().pull().unpin(1.0)

# add multiple sheets stacked with rotation
n, space = 5, 0.05
for i in range(n):
    y = (i + 1) * space
    deg = i * 90 / n
    obj = scene.add("sheet").direction([1, 0, 0], [0, 1, 0])
    obj.at(0, y, 0).rotate(90, "x").rotate(deg + 5 * random.random(), "y")
    obj.param.set("strain-limit", 0.05).set("friction", 0.5)

# compile the scene and report stats
scene = scene.build().report()

# preview the initial scene
scene.preview()

In [ ]:
# create a new session with the compiled scene
session = app.session.create(scene)

# set session parameters
session.param.set("frames", 120)

# build this session
session = session.build()

In [ ]:
# start the simulation and live-preview the results
session.start().preview()

# also show simulation logs in realtime
session.stream()

In [ ]:
# create an animation from the simulation results
session.animate()

In [ ]:
# export the animation to file
session.export.animation()

In [ ]:
# this is for CI
if app.ci:
    assert session.finished()